# Project 1 - Locations of charging stations for battery electric vehicles.

## Introduction to the project

### Objective

### Data set

#### Index sets

$dis\_idx \in DIS\_IDX$: Index and set of districts.

$fs\_idx \in FS\_IDX = [0, N\_FIXED]$: Index and set of fixed rental stations.

$ds\_idx \in DS\_IDX = [0,N\_DYNAMIC*3]$: Index and set of dynamic rental stations.

$s\_idx \in S\_IDX = fs\_idx \cup ds\_idx$: Index and set of dynamic rental stations.


$\quad \forall ds\_idx \in DS\_IDX$: Type of $ds\_idx$ = $ds\_idx$ // $N\_DYNAMIC$ (0:'small', 1:'medium', 2:'large')

$\quad \forall ds\_idx \in DS\_IDX$: Position of $ds\_idx$ = $ds\_idx$ % $N\_DYNAMIC$.



#### Parameters

$demand_{dis\_idx} \in \mathbb{R}^+$: Demand for stations in district $dis\_idx$.

$\text{costS} = 10$: Cost of building a small station.

$\text{costM} = 20$: Cost of building a medium station.

$\text{costL} = 50$: Cost of building a large station.

$capS= 10$: Number of bikes for a small station.
 
$capM = 30$: Number of bikes for a medium station.

$capL = 100$: Number of bikes for a large station.

#### Decision variables

$ysmall_{ds\_idx} \in \{0, 1 \}$: 1 if small station built at position of $ds\_idx$; otherwise 0.

$ymed_{ds\_idx} \in \{0, 1 \}$: 1 if medium station built at position of $ds\_idx$; otherwise 0.

$ylarge_{ds\_idx} \in \{0, 1 \}$: 1 if large station built at position of $ds\_idx$; otherwise 0.

$x_{s\_idx,dis\_idx} \in \mathbb{R}^+$: Amount of the district's $dis\_idx$ demand met by rental station $s\_idx$. 

#### Objective function

- **Costs**. Minimize investment costs.
 

\begin{equation}
\text{Min} \quad CostTarget = 
\text{costS}\sum_{i = 0}^{N\_DYNAMIC} ysmall_i + \text{costM}\sum_{i = N\_DYNAMIC}^{N\_DYNAMIC * 2} ymed_i + \text{costL}\sum_{i = N\_DYNAMIC * 2}^{N\_DYNAMIC * 3} ylarge_i
\tag{0}
\end{equation}


- **Distance**. Minimize Distance from District to Stations supplying their demand.
\begin{equation}
\text{Min} \quad DistanceTarget = \sum_{s\_idx \in S\_IDX} \sum_{dis\_idx \in DIX\_IDX} x_{s\_idx, dis\_idx} * ||pos(s\_idx) - pos(dis\_idx)||
\tag{1}
\end{equation}


- **Combined**. Combine Targets.
\begin{equation}
\text{Min} \quad T = w1*CostTarget + w2*DistanceTarget
\end{equation}

#### Constraints

- **Demand**. The complete demand of a district $dis\_idx$ should be met.

\begin{equation}
\sum_{s\_idx  \in  S\_IDX} x_{s\_idx,dis\_idx} = demand_{dis\_idx} \quad \forall dis\_idx \in DIS\_IDX
\tag{1}
\end{equation}

- **Capacity**. Capacity of stations cannot be exceeded. 
\begin{equation}
\sum_{dis\_idx \in DIS\_IDX} x_{s\_idx,dis\_idx} \leq ysmall_{s\_idx}*capS + ymed_{s\_idx}*capM + ylarge_{s\_idx}*capL  \quad \forall s\_idx \in S\_IDX
\tag{2}
\end{equation}

- **Physics**. At every possible location $ds\_idx$ a maximum of one charging station can be built. 

\begin{equation}
ysmall_{ds\_idx} + ymed_{ds\_idx} + ylarge_{ds\_idx} \leq 1 \quad \forall ds\_idx \in DS\_IDX
\tag{3}
\end{equation}

# Implementation

In [1]:
from math import sqrt
import ipywidgets as widgets
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def prep_data(dis_trunc, fs_trunc, n_dynamic):
    date = "/".join(map(lambda x: "0"+str(x.value) if x.value<10 else str(x.value), date_selector.children))
    # fixed stations
    coords = pd.read_csv("processed_data/koordinaten.csv").drop(columns="Station").rename(columns={"Terminal": "Station"})
    df_fs = pd.read_csv("processed_data/station_demand_per_day.csv", low_memory=False)
    df_fs["Station"] = pd.to_numeric(df_fs["Station"], errors="coerce", downcast="integer")
    df_fs = pd.merge(df_fs, coords, on="Station")
    df_fs = df_fs.where(df_fs["Day"] == date).dropna().drop(columns=["Day", "Docks", "Demand"])
    df_fs.rename(columns={"Longitude": "Lon", "Latitude": "Lat"}, inplace=True)
    df_fs["Station"] = range(len(df_fs))
    # districts
    zipcoords = pd.read_csv("processed_data/zipcoords.csv", header=None, names=["District", "Lat", "Lon"])
    df_dis = pd.merge(pd.read_csv("processed_data/district_demand_per_day.csv"), zipcoords, on="District")
    df_dis = df_dis.where(df_dis["Day"] == date).dropna().drop(columns=["Day"])
    # dynamic stations
    df_ds = pd.DataFrame(columns=["Station", "Lat", "Lon"])
    for i in range(n_dynamic):
        df_ds.loc[i] = [int(i), np.random.uniform(*df_dis["Lat"].agg(["min", "max"])), np.random.uniform(*df_dis["Lon"].agg(["min", "max"]))]
    df_ds["Station"] = pd.to_numeric(df_ds["Station"], errors="coerce", downcast="integer")

    # truncate to fit license restrictions
    return df_dis.head(dis_trunc), df_fs.head(fs_trunc), df_ds

In [3]:
def draw_built_stations(solution, df_dis, df_fs, df_ds):
    typed_ds = df_ds.copy()
    typed_ds["Type"] = solution["stations built"].map({
        0:"Ignored",
        1: "Built Small",
        2: "Built Medium",
        3: "Built Large"
    })
    typed_fixed = df_fs.copy()
    typed_fixed["Type"] = "Prebuilt"
    typed_distr = df_dis.copy()
    typed_distr["Type"] = "District"
    # create scatter trace for nodes
    return px.scatter_mapbox(pd.concat([typed_ds, typed_fixed, typed_distr]), lat="Lat", lon="Lon", color="Type",color_discrete_map={
            "Ignored": "black",
            "Prebuilt": "orange",
            "Built Small": "lightgreen",
            "Built Medium": "green",
            "Built Large": "darkgreen",
            "District": "blue"}, mapbox_style='open-street-map', zoom=11, center={"lat": df_dis["Lat"].mean(), "lon": df_dis["Lon"].mean()})


In [4]:
def solve(df_dis, df_fs, df_ds):
    """Solve MILP of station assignment.

    Args:
        df_dis: dataframe with districts and demand
        df_fs: dataframe with fixed stations 
        df_ds: dataframe with dynamic stations

    Returns:
        output dict: Solution of the MILP
    """
    # Costs
    costS, costM, costL = list(map(lambda x: x.value, costs_box.children))

    # Capacities
    capS, capM, capL = list(map(lambda x: x.value, capacities_box.children))

    demandModifier = demandModifier_box.value

    fs = {s:(x,y) for _, (s,x,y) in df_fs.iterrows()}
    ds = {s:(x,y) for _, (s,x,y) in df_ds.iterrows()}

    # district to coord
    distr = {d:(x,y) for _, (d,x,y,_) in df_dis.iterrows()}

    # station to capacity (fixed for now, didn't find data @dataacquisition O.o)
    capacity = {s:capM for _, (s,_,_) in df_fs.iterrows()}
    

    # district to demand
    demand = {d:dem*demandModifier for _, (d,dem,_,_) in df_dis.iterrows()}

    n_fixed = len(df_fs)
    n_dynamic = len(df_ds)

    # Loss weighting
    w1, w2 = list(map(lambda x: x.value, lossweighting_box.children))

    # Calculation of the distances between the districts and the locations of the charging stations
    dists_fix = {}
    for d_i, d_v in distr.items():
        for s_i, s_v in fs.items():
            dists_fix[(d_i, s_i)] = sqrt((d_v[0] - s_v[0])**2 + (d_v[1] - s_v[1])**2)

    dists_dyn = {}
    for d_i, d_v in distr.items():
        for s_i, s_v in ds.items():
            dists_dyn[(d_i, s_i)] = sqrt((d_v[0] - s_v[0])**2 + (d_v[1] - s_v[1])**2)

    #####################################################
    #               MILP Model Formulation              #
    #####################################################

    # Model
    m = gp.Model('eval_rental_positions')

    # Decision variables
    offsetidx = [range(0, n_dynamic), range(n_dynamic, n_dynamic*2), range(n_dynamic*2, n_dynamic*3)]
    ys = m.addVars(offsetidx[0], vtype=GRB.BINARY, name='small')
    ym = m.addVars(offsetidx[1], vtype=GRB.BINARY, name='medium')
    yl = m.addVars(offsetidx[2], vtype=GRB.BINARY, name='large')

    # Parameter
    xdyn = m.addVars(dists_dyn.keys(), vtype=GRB.CONTINUOUS, name='AssignDyn', lb=0)
    xfix = m.addVars(dists_fix.keys(), vtype=GRB.CONTINUOUS, name='AssignFix', lb=0)

    # Objective function
    m.setObjective(
        w1 * (costS * gp.quicksum(ys[i] for i in offsetidx[0]) + costM * gp.quicksum(ym[i] for i in offsetidx[1]) + costL * gp.quicksum(yl[i] for i in offsetidx[2])) 
        +
        gp.quicksum(w2 * xdyn[(i,j)]*dists_dyn[(i,j)] for i in distr.keys() for j in range(n_dynamic)) 
        +
        gp.quicksum(w2 * xfix[(i,j)]*dists_fix[(i,j)] for i in distr.keys() for j in range(n_fixed))
        ,
        GRB.MINIMIZE
    )
    # dynamic and fixed stations supply all demand
    m.addConstrs((gp.quicksum(xdyn[(d,j)] for j in range(n_dynamic)) + gp.quicksum(xfix[(d,j)] for j in range(n_fixed)) >= demand[d] for d in distr.keys()), name="demandConstr")
    # dynamic stations have enough capacity
    m.addConstrs((gp.quicksum(xdyn[(d,i)] for d in distr.keys()) <= ys[i]*capS + ym[j]*capM + yl[k]*capL for (i,j,k) in zip(*offsetidx)), name="capacityFixConstr")
    # fixed stations have enough capacity
    m.addConstrs((gp.quicksum(xfix[(i,j)] for i in distr.keys()) <= capacity[j] for j in range(n_fixed)), name="capacityDynConstr")
    # max one station per location
    m.addConstrs((ys[i] + ym[j] + yl[k] <= 1 for (i,j,k) in zip(*offsetidx)), name="PhysicsConstr")

    # non verbose
    m.setParam('OutputFlag', 0)
    m.optimize()
    if m.status == GRB.INFEASIBLE:
        raise Exception("Infeasible model")
    stations_built = pd.Series(df_ds.index).apply(lambda x: ys[x].X+2*ym[x+n_dynamic].X+3*yl[x+2*n_dynamic].X)
    getvals = lambda x: {k: v.x for k, v in x.items() if v.x > 0}
    return {
        "status": m.status,
        "objective function": m.objVal,
        "runtime": m.Runtime,
        "demand to satisfy": sum(demand.values()) - sum(capacity.values()),
        "new supply built": sum(ys[i].x*capS + ym[j].x*capM + yl[k].x*capL for (i,j,k) in zip(*offsetidx)),
        "dynamic assignment": getvals(xdyn),
        "fixed assignment": getvals(xfix),
        "stations built": stations_built,
    }

In [5]:
# repeatedly defining widgets breaks correspendence between widgets and their values
n_dynamic_slider = widgets.IntSlider(30, min=5, max=50, step=1, description='n dynamic')
costs_box = widgets.HBox([widgets.IntText(5, description='cost small'), widgets.IntText(10, description='cost medium'), widgets.IntText(40, description='cost large')])
capacities_box = widgets.HBox([widgets.IntText(5, description='capacity small'), widgets.IntText(15, description='capacity medium'), widgets.IntText(50, description='capacity large')])
lossweighting_box = widgets.HBox([widgets.FloatText(1, description='loss weight 1'), widgets.FloatText(1.5, description='loss weight 2')])
demandModifier_box = widgets.FloatSlider(1.3, min=0.5, max=2.9, step=0.1, description='demand modifier')
date_selector = widgets.HBox([widgets.IntText(1, description='day'), widgets.IntText(1, description='month'), widgets.IntText(2019, description='year')])
display(widgets.VBox([n_dynamic_slider, date_selector, costs_box, capacities_box, lossweighting_box, demandModifier_box]))

In [6]:
data = prep_data(dis_trunc=30, fs_trunc=15, n_dynamic=n_dynamic_slider.value)
solution = solve(*data)
draw_built_stations(solution, *data)

Restricted license - for non-production use only - expires 2024-10-28


In [7]:
solution

{'status': 2,
 'objective function': 140169.35746750387,
 'runtime': 0.007999897003173828,
 'demand to satisfy': 521.2000000000003,
 'new supply built': 525.0,
 'dynamic assignment': {('M5G 1', 25.0): 0.40000000000000213,
  ('M5G 1', 27.0): 11.299999999999999,
  ('M4Y 2', 1.0): 4.599999999999984,
  ('M4Y 2', 4.0): 15.0,
  ('M5B 2', 12.0): 15.0,
  ('M5B 2', 17.0): 15.0,
  ('M5B 2', 19.0): 2.8000000000000043,
  ('M5B 2', 20.0): 13.299999999999997,
  ('M5E 1', 5.0): 7.1,
  ('M5G 2', 9.0): 12.700000000000003,
  ('M5G 2', 25.0): 1.5999999999999979,
  ('M5C 0', 15.0): 15.0,
  ('M5C 0', 20.0): 1.7000000000000028,
  ('M5C 0', 26.0): 15.0,
  ('M5C 0', 29.0): 15.0,
  ('M4W 0', 3.0): 11.100000000000012,
  ('M4W 0', 11.0): 10.99999999999999,
  ('M5V 3', 5.0): 7.9,
  ('M5V 3', 8.0): 1.1999999999999993,
  ('M5G 0', 2.0): 12.40000000000001,
  ('M5G 0', 18.0): 8.399999999999991,
  ('M5A 3', 24.0): 15.0,
  ('M7A 1', 2.0): 20.8,
  ('M5J 0', 14.0): 9.900000000000034,
  ('M5J 0', 16.0): 15.0,
  ('M6J 1', 